In [30]:
import dolfinx as dfx
from mpi4py.MPI import COMM_WORLD as comm
import ufl

## Mesh and function space

In [31]:
# Mesh
n_elem = 16
mesh = dfx.mesh.create_unit_interval(comm, n_elem)

# Elements
num_particles = 96

elem1 = ufl.FiniteElement("Lagrange", mesh.ufl_cell(), 1)

elem_c = elem1
elem_mu = elem1

multi_particle_element = ufl.MixedElement(
    [
        [
            elem_c,
        ]
        * num_particles,
        [
            elem_mu,
        ]
        * num_particles,
    ]
)

# Function space
V = dfx.fem.FunctionSpace(mesh, multi_particle_element)
V_c = dfx.fem.FunctionSpace(mesh, elem_c)

## Grid info and function

In [32]:
coords = ufl.SpatialCoordinate(mesh)
r2 = ufl.dot(coords, coords)

u = dfx.fem.Function(V)

In [33]:
c, mu = u.split()

cs = c.split()

c0 = dfx.fem.Function(V_c)
c1 = dfx.fem.Function(V_c)

## Timing

Compare compilation of every individual particle form vs. replacing expresion in prototype.

In [34]:
%%timeit
cs_forms = [dfx.fem.form(3 * r2 * c_ * ufl.dx) for c_ in cs]

1.22 s ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit

c_form_prototype = 3 * r2 * c0 * ufl.dx

cs_forms = [dfx.fem.form(ufl.replace(c_form_prototype, {c0: c_})) for c_ in cs]

1.22 s ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
